In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader as dl
import pandas as pd
import numpy as np

In [2]:
# Import data #TODO: add data to github and make this a local import
path = '/mnt/c/Users/han/Downloads/merged_obs_sim_features_ref_plus_dist.csv'
raw = pd.read_csv(path)

In [3]:
# Extract relevant columns
x = raw.values[:,[3,4,5,6,7]]
y = raw.values[:,10]
feature_labels = raw.columns[[4,5,6,7]]

# Add features
temp_ind = raw.columns.get_loc('vaisala_temp')
temp_col = raw.values[:,temp_ind].reshape(-1,1)
x = np.hstack((x,temp_col**2))
x = np.hstack((x, temp_col**3))
print(x.shape)

(31635, 7)


In [21]:
# 0-mean, same variance
m = np.mean(x,0)
m = m.reshape(1,-1)
print(m.shape)
x_mean =  x - m

for i, feat in enumerate(x_mean.T):
    s = np.std(feat)
    x_mean[:,i] = feat/s
    print(np.std(x_mean[:,i]))

print(x_mean.shape)

(1, 7)
1.0000000000000036
1.0000000000000049
1.0000000000000029
1.000000000000366
0.9999999999999389
0.9999999999999989
0.9999999999999962
(31635, 7)


In [22]:
# Split data
data = x_mean
train_size = int(.8 *len(data))
test_size = len(data) - train_size
val_size = int(.25*train_size)
print("Train: ",train_size," Test: ",test_size," Val: ",val_size)
X_train, X_test = torch.utils.data.random_split(data, [train_size,test_size])
print(len(X_train),len(X_test))
X_train, X_val  = torch.utils.data.random_split(X_train, [train_size-val_size, val_size])
print(len(X_train),len(X_val))

Train:  25308  Test:  6327  Val:  6327
25308 6327
18981 6327


In [23]:
y_train = y[X_train.indices]
y_test  = y[X_test.indices]
y_val   = y[X_val.indices]

In [24]:
# Save data to files:
na = np.array(X_train.dataset.dataset[X_train.indices]).astype(np.float32)
np.save('/home/han/repos/BEACO2N-CO2-Correction/data/X_train', na)

na = np.array(X_test.dataset[X_test.indices]).astype(np.float32)
np.save('/home/han/repos/BEACO2N-CO2-Correction/data/X_test', na)

na = np.array(X_val.dataset.dataset[X_val.indices]).astype(np.float32)
np.save('/home/han/repos/BEACO2N-CO2-Correction/data/X_val', na)

na = np.array(y_train).astype(np.float32)
np.save('/home/han/repos/BEACO2N-CO2-Correction/data/y_train', na)

na = np.array(y_test).astype(np.float32)
np.save('/home/han/repos/BEACO2N-CO2-Correction/data/y_test', na)

na = np.array(y_val).astype(np.float32)
np.save('/home/han/repos/BEACO2N-CO2-Correction/data/y_val', na)

In [159]:
np.mean(X_train,0)

array([0.01789793479552376, 0.02399428198203704, 0.011346996285701223,
       0.00017953742198479255, -0.0008217586024538616, 0.1748679984563908,
       0.4105448534992399], dtype=object)

In [102]:
np.array(y_train).astype(np.float32).shape

(18981,)

In [20]:
np.std(x_mean[:,2])

6.936350461162214